# Interpreting Existing models - examples

In [1]:
import pandas as pd
import sklearn
import shap

#### We'll start by importing our dataset below. Unlike last time, this time we'll load it from an external datafile

In [2]:
df = pd.read_excel('https://github.com/datawhys/demo-xavier-ai-summit/blob/main/asthma_data.xlsx?raw=true')

In [3]:
df

,Dermatitis to Bronchiale,Gender,Race,Age of Mother,Age of Father,Pregnancy Duration,Rank in Family,Delivery Method,Birth Weight,Duration Breast Fed exclusive in weeks,Vaccination Mumps,Exposure to House Dust Mites (antibody),Exposure to Grass Pollen (antibody),Exposure to Egg (antibody),Exposure to Dairy (antibody),Exposure to Mold and Mildew
0,2. Asthma,Male,African/american,40.0,47.0,38,4,Caesarean,2.500,65.0,No,80.000,0.790,5.070,1.400,80.000
1,2. Asthma,Female,African/american,17.0,22.0,38,1,Natural,2.800,2.0,No,80.000,0.175,29.800,4.030,80.000
2,1. No Asthma,Female,African/american,28.0,30.0,38,2,Natural,2.920,13.0,No,35.200,0.175,0.175,0.175,35.900
3,2. Asthma,Male,African/american,17.0,19.0,38,2,Natural,3.325,9.0,Yes,0.175,0.520,10.300,18.500,21.110
4,1. No Asthma,Male,African/american,31.0,32.0,40,1,Natural,3.500,4.0,No,2.610,0.175,80.000,44.600,7.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1. No Asthma,Female,Other/mixed race,34.0,38.0,38,2,Natural,3.750,19.0,Yes,1.620,0.175,1.030,3.070,2.320
396,1. No Asthma,Male,Other/mixed race,21.0,24.0,40,2,Natural,3.240,NaN,No,38.000,0.175,6.580,0.460,NaN
397,2. Asthma,Female,Other/mixed race,24.0,29.0,41,1,Natural,3.700,13.0,Yes,NaN,0.175,0.175,0.175,NaN
398,1. No Asthma,Male,Other/mixed race,27.0,29.0,36,1,Caesarean,2.220,NaN,No,1.490,0.175,0.175,NaN,NaN
